In [1]:
import gym
import torch
from torch import nn
from collections import namedtuple, deque
import itertools
from copy import deepcopy
import random
import numpy as np
from torch.utils.tensorboard import SummaryWriter
import tqdm
from math import log

/home/devjac/Code/python/learn-pytorch/.venv/lib/python3.10/site-packages/gym/wrappers/monitoring/video_recorder.py:9: DeprecationWarning: The distutils package is deprecated and slated for removal in Python 3.12. Use setuptools or check PEP 632 for potential alternatives
  import distutils.spawn


In [2]:
RENDER = False

In [3]:
env = gym.make("LunarLander-v2")

In [4]:
# How much we value entropy / exploration.
ALPHA_TARGET = 0.8 * log(env.action_space.n)            
GAMMA = 1 - 0.01        # How much we value future rewards.
TAU = 0.01              # How much q_target is updated when polyak averaging (step 15).
POLICY_LR = 0.001       # Policy learning rate.
Q_LR = 0.001            # Q learning rate.
ALPHA_LR = 0.001        # ALPHA learning rate.

In [5]:
SARS = namedtuple('SARS', 'state, action, reward, next_state, t, failed, limit')

In [6]:
softmax = nn.Softmax(dim=0)
input = torch.tensor([1, 2, 3], dtype=float)
display(input)
output = softmax(input)
display(output)
sum(output)

tensor([1., 2., 3.], dtype=torch.float64)

tensor([0.0900, 0.2447, 0.6652], dtype=torch.float64)

tensor(1.0000, dtype=torch.float64)

In [7]:
softmax = nn.Softmax(dim=1)
input = torch.tensor([[1, 2, 3], [1, 2, 3], [3, 3, 3]], dtype=float)
display(input)
output = softmax(input)
display(output)
sum(output)

tensor([[1., 2., 3.],
        [1., 2., 3.],
        [3., 3., 3.]], dtype=torch.float64)

tensor([[0.0900, 0.2447, 0.6652],
        [0.0900, 0.2447, 0.6652],
        [0.3333, 0.3333, 0.3333]], dtype=torch.float64)

tensor([0.5134, 0.8228, 1.6638], dtype=torch.float64)

In [8]:
class PolicyNetwork(nn.Module):
    def __init__(self, input_size, output_size):
        super().__init__()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(input_size, 2000),
            nn.ReLU(),
            nn.Linear(2000, 1500),
            nn.ReLU(),
            nn.Linear(1500, output_size)
        )

    def forward(self, x):
        nn_out = self.linear_relu_stack(x)
        return nn.Softmax(dim=1)(nn_out)

    def __call__(self, x):
        raise RuntimeError("Use forward")

In [9]:
policy_network = PolicyNetwork(4, 2)
policy_network

PolicyNetwork(
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=4, out_features=2000, bias=True)
    (1): ReLU()
    (2): Linear(in_features=2000, out_features=1500, bias=True)
    (3): ReLU()
    (4): Linear(in_features=1500, out_features=2, bias=True)
  )
)

In [10]:
torch.cuda.is_available()

True

In [11]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [12]:
mock_states = torch.rand(5, 4)
mock_states

tensor([[0.7214, 0.7215, 0.3766, 0.8725],
        [0.9382, 0.8720, 0.7283, 0.2236],
        [0.5513, 0.5107, 0.4888, 0.5453],
        [0.2224, 0.6237, 0.0639, 0.5532],
        [0.7662, 0.4477, 0.6061, 0.4348]])

In [13]:
policy_network.forward(mock_states)

tensor([[0.5189, 0.4811],
        [0.5126, 0.4874],
        [0.5140, 0.4860],
        [0.5109, 0.4891],
        [0.5130, 0.4870]], grad_fn=<SoftmaxBackward0>)

In [14]:
class QNetwork(nn.Module):
    def __init__(self, input_size, output_size):
        super().__init__()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(input_size, 2000),
            nn.ReLU(),
            nn.Linear(2000, 1500),
            nn.ReLU(),
            nn.Linear(1500, output_size)
        )

    def forward(self, x):
        nn_out = self.linear_relu_stack(x)
        return nn_out
    
    def __call__(self, x):
        raise RuntimeError("Use forward")

In [15]:
q_network = QNetwork(4, 2)
q_network

QNetwork(
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=4, out_features=2000, bias=True)
    (1): ReLU()
    (2): Linear(in_features=2000, out_features=1500, bias=True)
    (3): ReLU()
    (4): Linear(in_features=1500, out_features=2, bias=True)
  )
)

In [16]:
q_network.forward(mock_states)

tensor([[0.0485, 0.0901],
        [0.0807, 0.1153],
        [0.0455, 0.0862],
        [0.0277, 0.0566],
        [0.0541, 0.1082]], grad_fn=<AddmmBackward0>)

In [17]:
def run_episode(action_f, step_f, env, policy, fail_at_limit=False):
    episode_reward = 0
    s = env.reset()
    for t in itertools.count(start=1):
        a = action_f(policy, s)
        next_state, reward, failed, info = env.step(a)
        episode_reward += reward
        assert t <= env._max_episode_steps
        limit = t == env._max_episode_steps
        if limit and not fail_at_limit:
            failed = False
        assert fail_at_limit or not (limit and failed)
        step_f(s, a, reward, next_state, t, failed, limit)
        if failed or limit:
            break
        s = next_state
    return episode_reward

In [18]:
class Policy:
    def __init__(self, env_state_size, env_action_space_size):
        self.policy_network = PolicyNetwork(env_state_size, env_action_space_size)
        self.q1_network = QNetwork(env_state_size, env_action_space_size)
        self.q2_network = QNetwork(env_state_size, env_action_space_size)
        self.q1_target_network = deepcopy(self.q1_network)
        self.q2_target_network = deepcopy(self.q2_network)
        self.alpha = torch.tensor(ALPHA_TARGET, dtype=float, requires_grad=True)
        self.policy_network.to(device)
        self.q1_network.to(device)
        self.q2_network.to(device)
        self.q1_target_network.to(device)
        self.q2_target_network.to(device)
        self.alpha.to(device)
        self.reset_optimizers()

    def reset_optimizers(self):
        self.policy_optimizer = torch.optim.SGD(self.policy_network.parameters(), lr=POLICY_LR)
        self.q1_optimizer = torch.optim.SGD(self.q1_network.parameters(), lr=Q_LR)
        self.q2_optimizer = torch.optim.SGD(self.q2_network.parameters(), lr=Q_LR)

    @property
    def alpha_dc(self):
        """Alpha, (D)etached and (C)lamped"""
        return self.alpha.detach().clamp(min=0)

In [19]:
t = torch.tensor(1, dtype=float, requires_grad=True)
display(t)
torch.optim.SGD([t], lr=ALPHA_LR)

tensor(1., dtype=torch.float64, requires_grad=True)

SGD (
Parameter Group 0
    dampening: 0
    lr: 0.001
    maximize: False
    momentum: 0
    nesterov: False
    weight_decay: 0
)

In [20]:
replay_buffer = deque(maxlen=30_000)

In [21]:
env.action_space.n

4

In [22]:
env.observation_space.shape

(8,)

In [23]:
oss = env.observation_space.shape
if len(oss) != 1:
    raise RuntimeError(f'Unknown observation_space.shape: {oss}')
os_len = oss[0]
policy = Policy(os_len, env.action_space.n)

In [24]:
s = env.reset()
s

array([-0.00504427,  1.4097033 , -0.51096046, -0.0540935 ,  0.00585198,
        0.11574016,  0.        ,  0.        ], dtype=float32)

In [25]:
s = torch.tensor(s).reshape((1, -1))
s

tensor([[-0.0050,  1.4097, -0.5110, -0.0541,  0.0059,  0.1157,  0.0000,  0.0000]])

In [26]:
policy_output = policy.policy_network.forward(s.to(device))
policy_output

tensor([[0.2637, 0.2348, 0.2614, 0.2402]], device='cuda:0',
       grad_fn=<SoftmaxBackward0>)

In [27]:
action_weights = policy_output.reshape((-1,)).tolist()

In [28]:
list(range(4))

[0, 1, 2, 3]

In [29]:
action = random.choices(range(len(action_weights)), weights=action_weights)[0]
action

3

In [30]:
def action(policy, s):
    tensor_s = torch.tensor(s).reshape((1, -1)).to(device)
    action_weights = policy.policy_network.forward(tensor_s).reshape((-1,)).tolist()
    action = random.choices(range(len(action_weights)), weights=action_weights)[0]
    return action

def step(initial_s, a, r, next_s, t, failed, limit):
    replay_buffer.append(SARS(initial_s, a, r, next_s, t, failed, limit))
    if RENDER:
        env.render()

In [31]:
run_episode(action, step, env, policy)

-142.5046586585035

In [32]:
len(replay_buffer)

137

In [33]:
replay_buffer

deque([SARS(state=array([-0.00341415,  1.4199383 , -0.34582967,  0.4008054 ,  0.00396292,
               0.07833554,  0.        ,  0.        ], dtype=float32), action=0, reward=0.7184532415034539, next_state=array([-0.00682869,  1.4283781 , -0.34536165,  0.3750935 ,  0.00783391,
               0.0774275 ,  0.        ,  0.        ], dtype=float32), t=1, failed=False, limit=False),
       SARS(state=array([-0.00682869,  1.4283781 , -0.34536165,  0.3750935 ,  0.00783391,
               0.0774275 ,  0.        ,  0.        ], dtype=float32), action=0, reward=0.7514958865412495, next_state=array([-0.01024332,  1.4362193 , -0.34537405,  0.34847087,  0.01170254,
               0.07737967,  0.        ,  0.        ], dtype=float32), t=2, failed=False, limit=False),
       SARS(state=array([-0.01024332,  1.4362193 , -0.34537405,  0.34847087,  0.01170254,
               0.07737967,  0.        ,  0.        ], dtype=float32), action=3, reward=1.4201871939735338, next_state=array([-0.01359787,  1.443

# Polyak Averaging

In [34]:
test_parameter_1 = next(policy.policy_network.named_parameters())[1]
test_parameter_1

Parameter containing:
tensor([[ 0.0869,  0.2175, -0.2735,  ...,  0.3220,  0.1154,  0.1878],
        [ 0.1990,  0.3471, -0.2968,  ..., -0.2157,  0.0724,  0.1855],
        [-0.1839, -0.2129, -0.2688,  ..., -0.2809, -0.0641, -0.3218],
        ...,
        [ 0.0303,  0.1337, -0.0363,  ..., -0.1207,  0.2753,  0.2580],
        [ 0.3386, -0.3390,  0.0697,  ..., -0.2697, -0.2651, -0.3351],
        [ 0.2313,  0.1664, -0.1894,  ..., -0.1986,  0.1208,  0.0946]],
       device='cuda:0', requires_grad=True)

In [35]:
test_parameter_2 = test_parameter_1 * 0 + 0.0128
test_parameter_2

tensor([[0.0128, 0.0128, 0.0128,  ..., 0.0128, 0.0128, 0.0128],
        [0.0128, 0.0128, 0.0128,  ..., 0.0128, 0.0128, 0.0128],
        [0.0128, 0.0128, 0.0128,  ..., 0.0128, 0.0128, 0.0128],
        ...,
        [0.0128, 0.0128, 0.0128,  ..., 0.0128, 0.0128, 0.0128],
        [0.0128, 0.0128, 0.0128,  ..., 0.0128, 0.0128, 0.0128],
        [0.0128, 0.0128, 0.0128,  ..., 0.0128, 0.0128, 0.0128]],
       device='cuda:0', grad_fn=<AddBackward0>)

In [36]:
test_parameter_1 * 0.9 + test_parameter_2 * 0.1

tensor([[ 0.0795,  0.1971, -0.2448,  ...,  0.2910,  0.1051,  0.1703],
        [ 0.1804,  0.3136, -0.2659,  ..., -0.1928,  0.0664,  0.1683],
        [-0.1642, -0.1903, -0.2406,  ..., -0.2515, -0.0564, -0.2883],
        ...,
        [ 0.0285,  0.1216, -0.0314,  ..., -0.1073,  0.2490,  0.2335],
        [ 0.3060, -0.3038,  0.0640,  ..., -0.2415, -0.2373, -0.3003],
        [ 0.2094,  0.1510, -0.1692,  ..., -0.1774,  0.1100,  0.0864]],
       device='cuda:0', grad_fn=<AddBackward0>)

In [37]:
def polyak_update(network_to_update, target_network, tau=0.001):
    with torch.no_grad():
        for to_update, target in zip(network_to_update.parameters(), target_network.parameters()):
            to_update *= 1-tau
            to_update += target * tau

In [38]:
test_network_1 = QNetwork(5, 3)
test_network_2 = QNetwork(5, 3)
display(list(test_network_1.parameters())[0])
display(list(test_network_2.parameters())[0])
polyak_update(test_network_2, test_network_1, 0.1)
display(list(test_network_1.parameters())[0])
display(list(test_network_2.parameters())[0])

Parameter containing:
tensor([[-0.1968, -0.3133, -0.2435,  0.4103, -0.0227],
        [ 0.1414,  0.0650,  0.1183, -0.2331,  0.3288],
        [-0.3775, -0.1019, -0.2006, -0.2421, -0.1323],
        ...,
        [-0.3563,  0.0161, -0.2362, -0.3596, -0.1366],
        [-0.3776, -0.0514,  0.3088, -0.4080,  0.2523],
        [-0.2336,  0.2381,  0.3409,  0.4260, -0.1260]], requires_grad=True)

Parameter containing:
tensor([[-0.0745, -0.2795,  0.2351,  0.1722, -0.2248],
        [ 0.0416, -0.1377,  0.3076,  0.0011, -0.0033],
        [-0.1036, -0.1190,  0.1200, -0.2078,  0.3957],
        ...,
        [ 0.3820, -0.3246,  0.1918, -0.2696, -0.2560],
        [-0.3427, -0.3578, -0.1116, -0.1990,  0.0619],
        [ 0.1042, -0.0303,  0.1295, -0.2048,  0.0961]], requires_grad=True)

Parameter containing:
tensor([[-0.1968, -0.3133, -0.2435,  0.4103, -0.0227],
        [ 0.1414,  0.0650,  0.1183, -0.2331,  0.3288],
        [-0.3775, -0.1019, -0.2006, -0.2421, -0.1323],
        ...,
        [-0.3563,  0.0161, -0.2362, -0.3596, -0.1366],
        [-0.3776, -0.0514,  0.3088, -0.4080,  0.2523],
        [-0.2336,  0.2381,  0.3409,  0.4260, -0.1260]], requires_grad=True)

Parameter containing:
tensor([[-0.0867, -0.2829,  0.1872,  0.1960, -0.2045],
        [ 0.0515, -0.1174,  0.2887, -0.0223,  0.0299],
        [-0.1310, -0.1173,  0.0880, -0.2112,  0.3429],
        ...,
        [ 0.3082, -0.2905,  0.1490, -0.2786, -0.2440],
        [-0.3462, -0.3271, -0.0696, -0.2199,  0.0810],
        [ 0.0704, -0.0034,  0.1507, -0.1417,  0.0739]], requires_grad=True)

# Log Experiments

In [39]:
for p in [0.99, 0.9, 0.8, 0.6, 0.5]:
    logs = -np.log([p, 1-p])
    display(p, logs, sum(logs))

0.99

array([0.01005034, 4.60517019])

4.615220521841592

0.9

array([0.10536052, 2.30258509])

...

array([0.51082562, 0.91629073])

1.4271163556401456

0.5

array([0.69314718, 0.69314718])

1.3862943611198906

In [40]:
random.sample(replay_buffer, k=4)

[SARS(state=array([-0.00682869,  1.4283781 , -0.34536165,  0.3750935 ,  0.00783391,
         0.0774275 ,  0.        ,  0.        ], dtype=float32), action=0, reward=0.7514958865412495, next_state=array([-0.01024332,  1.4362193 , -0.34537405,  0.34847087,  0.01170254,
         0.07737967,  0.        ,  0.        ], dtype=float32), t=2, failed=False, limit=False),
 SARS(state=array([-0.19894381,  1.4246852 , -0.24884152, -0.43007538, -0.19863813,
         0.10147209,  0.        ,  0.        ], dtype=float32), action=2, reward=4.824518695462143, next_state=array([-0.20133634,  1.4159374 , -0.24356234, -0.38822225, -0.19429831,
         0.08679622,  0.        ,  0.        ], dtype=float32), t=64, failed=False, limit=False),
 SARS(state=array([-0.20368114,  1.4079274 , -0.23821905, -0.35551232, -0.19053921,
         0.07518221,  0.        ,  0.        ], dtype=float32), action=3, reward=-0.8874731921938224, next_state=array([-0.20595494,  1.3992976 , -0.22929266, -0.38330793, -0.18861617,
 

# Training

![Psudocode](sac_psudocode.png)

Source: https://spinningup.openai.com/en/latest/algorithms/sac.html#pseudocode

In [41]:
def q_min(q1, q2, states):
    
    def f(q):
        state_values = q.forward(states.to(device)).detach()
        return state_values
        
    return torch.minimum(*map(f, (q1, q2)))

In [42]:
def train(policy, replay_buffer):
    stats = {}
    # Step 11
    training_batch = random.sample(replay_buffer, k=min(len(replay_buffer), 100))
    # Prep
    states = torch.tensor(np.array([sars.state for sars in training_batch]), requires_grad=False).to(device)
    actions = torch.tensor(np.array([sars.action for sars in training_batch]), requires_grad=False).to(device)
    actions_hot = nn.functional.one_hot(actions, env.action_space.n).to(device)
    rewards = torch.tensor(np.array([sars.reward for sars in training_batch]), requires_grad=False).to(device)
    next_states = torch.tensor(np.array([sars.next_state for sars in training_batch]), requires_grad=False).to(device)
    fails = torch.tensor(np.array([sars.failed for sars in training_batch]), dtype=int, requires_grad=False).to(device)
    # Step 12
    next_action_probs = policy.policy_network.forward(next_states.to(device)).detach()
    assert not next_action_probs.requires_grad
    next_states_q_min = q_min(policy.q1_target_network, policy.q2_target_network, next_states)
    assert not next_states_q_min.requires_grad
    next_actions_q_min = torch.sum(next_states_q_min * next_action_probs, 1)
    assert not next_actions_q_min.requires_grad
    next_actions_entropy = torch.sum(next_action_probs * torch.log(next_action_probs), 1)
    assert not next_actions_entropy.requires_grad
    y = rewards + GAMMA * (1-fails) * (next_actions_q_min - policy.alpha_dc * next_actions_entropy)
    assert not y.requires_grad
    # Step 13
    for qi, q, opt in ((1, policy.q1_network, policy.q1_optimizer),
                       (2, policy.q2_network, policy.q2_optimizer)):
        assert not states.requires_grad
        assert not actions_hot.requires_grad
        q_state_action = torch.sum(q.forward(states.to(device)) * actions_hot, 1)
        assert q_state_action.requires_grad
        q_loss = torch.mean((q_state_action - y)**2)
        assert q_loss.requires_grad
        stats[f'train/q_loss_{qi}'] = q_loss.detach()
        assert q_loss.requires_grad
        opt.zero_grad()
        q_loss.backward()
        opt.step()
    # Step 14
    action_probs = policy.policy_network.forward(states)
    assert action_probs.requires_grad
    states_q_min = q_min(policy.q1_network, policy.q2_network, states)
    assert not states_q_min.requires_grad
    actions_q_min = torch.sum(states_q_min * action_probs, 1)
    assert actions_q_min.requires_grad
    actions_entropy = torch.sum(action_probs * torch.log(action_probs), 1)
    assert actions_entropy.requires_grad
    stats['train/entropy'] = -actions_entropy.detach().mean()
    stats['train/entropy_percent'] = -actions_entropy.detach().mean() / log(action_probs.shape[1])
    policy_loss = -1 * torch.mean(actions_q_min - policy.alpha_dc * actions_entropy)
    assert policy_loss.requires_grad
    stats['train/policy_loss'] = policy_loss.detach()
    policy.policy_optimizer.zero_grad()
    policy_loss.backward()
    policy.policy_optimizer.step()
    # Alpha Adjust
    assert policy.alpha.requires_grad
    assert action_probs.requires_grad
    policy.alpha.requires_grad_(True)
    alpha_optimizer = torch.optim.SGD([policy.alpha], lr=ALPHA_LR)
    alpha_loss = -1 * policy.alpha * (actions_entropy.detach().mean() + ALPHA_TARGET)
    assert alpha_loss.requires_grad
    stats['train/alpha_loss'] = alpha_loss.detach()
    alpha_optimizer.zero_grad()
    alpha_loss.backward()
    alpha_optimizer.step()
    policy.alpha.data.clamp_(min=0)
    stats['train/alpha'] = policy.alpha.detach()
    # Step 15
    polyak_update(policy.q1_target_network, policy.q1_network, tau=TAU)
    polyak_update(policy.q2_target_network, policy.q2_network, tau=TAU)
    return stats

In [43]:
tb_writer = SummaryWriter()

oss = env.observation_space.shape
if len(oss) != 1:
    raise RuntimeError(f'Unknown observation_space.shape: {oss}')
os_len = oss[0]
policy = Policy(os_len, env.action_space.n)

replay_buffer = deque(maxlen=30_000)

def action(policy, s):
    tensor_s = torch.tensor(s).reshape((1, -1))
    action_weights = policy.policy_network.forward(tensor_s.to(device)).reshape((-1,)).tolist()
    action = random.choices(range(len(action_weights)), weights=action_weights)[0]
    return action

def step(initial_s, a, r, next_s, t, failed, limit):
    replay_buffer.append(SARS(initial_s, a, r, next_s, t, failed, limit))
    if RENDER:
        env.render()

n_episodes = 4000
for episode in tqdm.tqdm(range(1, n_episodes+1)):
    episode_reward = run_episode(action, step, env, policy, fail_at_limit=True)
    tb_writer.add_scalar('main/episode_reward', episode_reward, episode)
    tb_writer.add_scalar('main/replay_buffer_length', len(replay_buffer), episode)
    tb_writer.add_scalar('main/alpha_target', ALPHA_TARGET, episode)
    tb_writer.add_scalar('main/alpha_target_percent', ALPHA_TARGET / log(env.action_space.n), episode)
    policy.reset_optimizers()
    for training_iteration in range(1, 100+1):
        stats = train(policy, replay_buffer)
        for stat, value in stats.items():
            tb_writer.add_scalar(stat, value, episode)
    if episode > n_episodes / 4:
        ALPHA_TARGET *= 0.5**(1/1500)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4000/4000 [56:49<00:00,  1.17it/s]


In [45]:
env = gym.make("LunarLander-v2")

def render_only(initial_s, a, r, next_s, t, failed, limit):
    env.render()

scores = []
for episode in range(1, 10+1):
    scores.append(run_episode(action, render_only, env, policy))

print(f'Mean score: {sum(scores) / len(scores):.0f}')

env.close()

Mean score: 289
